# Biomass spatial progation through HMRF

Local biomass estimates are spatially propagated through Hidden Markov Random Fields.
Hidden layer states are climation classification labels (e.g., Chave forest types or Holdridge life zones).
Cells of the grid are classified using elevation and precipitation data loaded from a raster file.

In [ ]:
import gdal
import numpy as np
import pandas as pd
import allometry

In [ ]:
# Raster file paths
alt_file = "/home/nelson/Documents/IDEAM/cust_layers/alt/alt.tif"
prec_file = "/home/nelson/Documents/IDEAM/cust_layers/precp/precp.tif"

alt_ras = gdal.Open(alt_file)
prec_ras = gdal.Open(prec_file)

transform = alt_ras.GetGeoTransform()
altXOrigin = transform[0]
altYOrigin = transform[3]
altPixelWidth = transform[1]
altPixelHeight = transform[5]

transform = prec_ras.GetGeoTransform()
precXOrigin = transform[0]
precYOrigin = transform[3]
precPixelWidth = transform[1]
precPixelHeight = transform[5]

# Raster data is loaded in memory as a numpy.array
alt_band = alt_ras.GetRasterBand(1)
alt_arr = alt_band.ReadAsArray(0,0,alt_ras.RasterXSize, alt_ras.RasterYSize)

prec_band = prec_ras.GetRasterBand(1)
prec_arr = prec_band.ReadAsArray(0,0,prec_ras.RasterXSize, prec_ras.RasterYSize)


In [ ]:
# Data containers are set based on the area extent. So far HMRF layers are square grids.
w_limit = -76.0
s_limit = 0.0
e_limit = -70.0
n_limit = 6.0

lon_bords = [w_limit + x * 0.5 for x in xrange(13)]
lat_bords = [s_limit + x * 0.5 for x in xrange(13)]

alt_cont = np.zeros((len(lat_bords) - 1, len(lon_bords) - 1)) 
prec_cont = np.zeros((len(lat_bords) - 1, len(lon_bords) - 1)) 
count_cont = np.zeros((len(lat_bords) - 1, len(lon_bords) - 1)) 

In [ ]:
# Environmental data arrays are and means are estimated for the extent of the grid cell
it = np.nditer((alt_arr, prec_arr), flags=['multi_index'])
while not it.finished:
    row = it.multi_index[0]
    col = it.multi_index[1]
    lon = col * altPixelWidth + altXOrigin
    lat = row * altPixelHeight + altYOrigin

    if w_limit < lon < e_limit and s_limit < lat < n_limit:
        xindx = int((lon - w_limit) / 0.5)
        yindx = int((lat - s_limit) / 0.5)
        
        if it[0] >= 0 and it[1] > 0:
            alt_cont[yindx][xindx] += it[0]
            prec_cont[yindx][xindx] += it[1]
            count_cont[yindx][xindx] += 1.0
                
    it.iternext()


alt_cont = np.divide(alt_cont, count_cont)
prec_cont = np.divide(prec_cont, count_cont)

In [ ]:
# Each cell of the grid is climatically classified
states = {}
sym_count = 0
hold_cont = []
for rowa,rowp in zip(alt_cont,prec_cont):
    for a,p in zip(rowa, rowp):
        myhold = allometry.holdridge_col(a,p)
        if not myhold in states:
            states[myhold] = sym_count
            sym_count += 1
        hold_cont.append(states[myhold])
hold_cont = np.array(hold_cont)
hold_cont = np.reshape(hold_cont, (len(lat_bords) - 1, len(lon_bords) - 1))

In [ ]:
# Biomass estimates from plot data are loaded into memory
agb_plots = pd.read_csv("biomass_all_20180118.csv")

chaveII = []
for lat0, lat1 in zip(lat_bords[:-1], lat_bords[1:]):
    chaveII.append([])
    for lon0, lon1 in zip(lon_bords[:-1], lon_bords[1:]):
        cell_mean = agb_plots[(agb_plots.Longitud < lon1) & (agb_plots.Longitud > lon0) & 
            (agb_plots.Latitud < lat1) & (agb_plots.Latitud > lat0)]['chaveII'].mean()
        if pd.notna(cell_mean):
            chaveII[-1].append(cell_mean)
        else:
            chaveII[-1].append(0.0)

chaveII = np.array(chaveII)

In [ ]:
# Simple HMRF class
# Basically holds a data container and a parameter fitting to optimize emission functions
from scipy.stats import norm

class hmrf(object):
    def __init__(self, obs, states, symbols):
        self.rows = obs.shape[0]
        self.cols = obs.shape[1]
        self.obs = obs
        self.states = states
        self.symbols = symbols # list of symbols in hidden layer
        self.parameters = {x:(None, None) for x in self.symbols}
        
          
    def fit_params(self):
        for state in self.symbols:
            trainobs = []
            for x in xrange(self.rows):
                for y in xrange(self.cols):
                    if self.states[x][y] == state and self.obs[x][y] > 0:
                        trainobs.append(self.obs[x][y])
            #print trainobs
            self.parameters[state] = norm.fit(trainobs)

In [ ]:
# Example of HMRF class instantiation and observassion (biomass values) propagation 
test = hmrf(chaveII, hold_cont, (0,))
test.fit_params()
test.parameters